In [116]:
#import tensorflow as tf

#from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

#from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# STBD DATA USE THESE TWO
#true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\StbdTesting_1404_002_FINAL.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB13.csv"

#true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_stbd_avgBelow75deg.csv"
true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd_FinalCleaned.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB18.csv"
prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\StbdTrueReference_038cleanedFinal.csv"


#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"


In [ ]:
# PORT DATA USE THESE TWO

#true_path_port = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_port_avgBelow75deg.csv"
true_path_port = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port_FinalCleaned.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTrainMB18.csv"
prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PortTrueReference_038cleanedFinal.csv"

In [ ]:
df_true=pd.read_csv(true_path_stbd)
#df_true=pd.read_csv(true_path_port)
df_prediction=pd.read_csv(prediction_path)


In [ ]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["PredictedDOA"]]
df_predictedDOA

In [ ]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference

In [ ]:
df_reference['AngleUncertainty'].max()

In [ ]:
df_reference['AngleUncertainty'].mean()

In [ ]:
df_reference['AngleUncertainty'].var()

In [ ]:
df_reference['DOA'].var()

In [ ]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

In [ ]:
theShape = df_comparison.shape
numRows= theShape[0]

In [ ]:
numCols = 4
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail", "2STDPassFail", "4STDPassFail", "6STDPassFail"}
df_results

In [ ]:
df_comparison = df_comparison.join(df_results)
df_comparison

In [ ]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
print(df_comparison['Delta'].var())
df_comparison

In [ ]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison['2STDPassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty'])
df_comparison['4STDPassFail'] = df_comparison['AbsDelta'] <= (2*0.02*df_comparison['AngleUncertainty'])
df_comparison['6STDPassFail'] = df_comparison['AbsDelta'] <= (3*0.02*df_comparison['AngleUncertainty'])
df_comparison

In [ ]:
PassCount = df_comparison['PassFail'].sum()
PassCount2STD = df_comparison['2STDPassFail'].sum()
PassCount4STD = df_comparison['4STDPassFail'].sum()
PassCount6STD = df_comparison['6STDPassFail'].sum()

PassCount, PassCount2STD, PassCount4STD, PassCount6STD

In [ ]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount2STD = (~df_comparison['2STDPassFail']).sum()
FailCount4STD = (~df_comparison['4STDPassFail']).sum()
FailCount6STD = (~df_comparison['6STDPassFail']).sum()

FailCount, FailCount2STD, FailCount4STD, FailCount6STD

In [ ]:
TotalCount = PassCount+FailCount
TotalCount2STD = PassCount2STD+FailCount2STD
TotalCount4STD = PassCount4STD+FailCount4STD
TotalCount6STD = PassCount6STD+FailCount6STD

TotalCount, TotalCount2STD, TotalCount4STD, TotalCount6STD

In [ ]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass2STD = 100*(PassCount2STD/TotalCount2STD)
PercentPass4STD = 100*(PassCount4STD/TotalCount4STD)
PercentPass6STD = 100*(PassCount6STD/TotalCount6STD)

PercentPass, PercentPass2STD, PercentPass4STD, PercentPass6STD

In [ ]:
print('Maximum error within 1 STD Deviation: ', 0.5*0.02*df_comparison['AngleUncertainty'].max(), 'degrees')
print('Minimun error within 1 STD Deviation: ', 0.5*0.02*df_comparison['AngleUncertainty'].min(), 'degrees')